In [3]:
import pandas as pd
import numpy as np
data = pd.read_csv('polarity_labeling.csv')
data.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,asin,date,product,review,score,user,CS_SS,target,feature,polarity
0,B00000JFOA,"01 6, 2000",c-2000,the newer model c-2020 has all the fixes exce...,4.0,Lee,CS,c-2020,lens,0
1,B000023VQC,"06 24, 2001",pdc-700,it likes to change from xga to vga etc.. *the ...,2.0,A. Gray,CS,vga,battery,0
2,B00003OPEY,"04 26, 2000",dsc-f505,the included memory stick is a little small b...,5.0,David Pirmann,CS,110,memory,1
3,B00004R8V6,"09 5, 2000",d-360l,it also adds a nifty little optical function ...,5.0,"Baltic Books ""Vic""",CS,d-460,"optical,battery,lcd,compression,flash",1
4,B00004R8VC,"04 18, 2000",d-460,i read a lot of reviews on the d450 optical be...,5.0,JR,CS,d450,optical,-1


In [4]:
def Make_matrix(df):
    # unique()는 특정 시리즈 데이터에서 유일한 값들을 넘파이배열로 출력
    product_list = df['product'].unique()
    matrix = np.zeros((len(product_list), len(product_list)), dtype=np.float32)
    
    # 불필요한 데이터 제거
    df = df[df['polarity'] != 0]
    for i, p in enumerate(product_list):
        df_re = df[df['product'] == p]
        df_cs = df_re[df_re['CS_SS'] == 'CS']
        df_ss = df_re[df_re['CS_SS'] == 'SS']
        
        if i%100 == 0:
            print(i)
        
        for j, t in enumerate(product_list):
            if i == j:
                # ss에 의한 웨이트 계산
                ss_data = df_ss['polarity'].value_counts()
                if len(ss_data.index) == 2:
                    matrix[i,j] = int(ss_data[ss_data.index == 1]) / int(ss_data[ss_data.index == -1])
                elif 1 not in ss_data.index:
                    matrix[i,j] = 0
                else:
                    matrix[i,j] = int(ss_data[ss_data.index == 1])

            else:
                # cs에 의한 웨이트 계산
                cs_data = df_cs[df_cs['target'].str.contains(t) == True]['polarity'].value_counts()
                if len(cs_data.index) == 2:
                    matrix[i,j] = int(cs_data[cs_data.index == 1]) / int(cs_data[cs_data.index == -1])
                elif 1 not in cs_data.index:
                    matrix[i,j] = 0
                else:
                    matrix[i,j] = int(cs_data[cs_data.index == 1])
    
    # 매트릭스를 데이터프레임형식으로 바꿈
    dff = pd.DataFrame(matrix, index = product_list, columns = product_list)
    dff.loc['sum'] = dff.sum()

    return dff

In [8]:
def Make_Rank(df, d = 0.85, MAX_ITERATIONS = 50, threshold = 0.0001):
    # sum = 0 인 열 전체를 지우는 함수
    df = df.T
    df=df[df['sum'] != 0].T
    
    product_list = df.columns
    
    # 초기 스코어 설정( 1 )
    score = np.ones(len(product_list))
    
    # 최대 반복 횟수 50 설정
    for iter in range(0,MAX_ITERATIONS):
        prev_score = np.copy(score)
        
        for i, p in enumerate(product_list):
            # 넘파이 배열의 사칙연산을 사용하여 for문을 줄임
            score[i] = (1-d) + d*sum(df.loc[p] / df.loc['sum'] * prev_score)

        # 반복 중 수치 변화가 일정 값 미만으로 떨어지면 코드 종료
        if np.sum(np.fabs(prev_score-score)) <= threshold:
            print('요기서 끝 :', iter)
            break
            
    normal_score = score / np.sqrt((np.sum(score**2)))
    dff = pd.DataFrame(normal_score, index = product_list)
    return dff

In [79]:
def Make_Feature_Rank_Labeling(df):
    f_list = ['pixel','lens','optical','memory','burst','battery','focus','LCD','compression','flash']
    product_list = df['product'].unique()
    result = pd.DataFrame(index = product_list)
    
    for f in f_list:
        dff = df[df.feature.str.contains(f)]
        matrix = Make_matrix(dff)
        matrix.to_excel('{}_matrix.xlsx'.format(f), index = False)
        score = Make_Rank(matrix)
        score.columns = [f]
        
        result = pd.merge(result, df, left_index = True, right_index = True, how = 'left')    
        
    result = result.fillna(0)
    return result